# XGBoost 20210926 GPU sweep
Trying after building GPU version locally

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = True

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_xgboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    # !pip install category_encoders
    # !pip install catboost
#     !pip install --upgrade -q lightgbm

    # lighgbm gpu compatible
    # !git clone --recursive https://github.com/Microsoft/LightGBM
    # ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    # !pip install cmake --upgrade
    # # !pip install sklearn --upgrade
    # !git clone --recursive https://github.com/dmlc/xgboost
    # %cd /content/xgboost
    # !mkdir build
    # %cd build
    # !cmake .. -DUSE_CUDA=ON
    # !make -j4
    # %cd /content/xgboost/python-package
    # !python setup.py install --use-cuda --use-nccl
    # !/opt/bin/nvidia-smi
    # !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
# import catboost
from sklearn.utils import resample
import sklearn.metrics

Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [7]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
#     datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = root/'models'
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'optuna_studies'
    altdatapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/')
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath, altdatapath]:
        pth.mkdir(exist_ok=True)

In [8]:

# n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'xgboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
#     'optuna_trials': 20,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202110_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for XGBoost on GPU with Optuna, based on Boruta-optimized Shapley-value (SHAP)-calculated dataset with 136 features. Hope is that feature selection ==> less overfitting ==> more learning possible",
    'config': exmodel_config,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
# train_source = datapath/'train.feather'
# df = pd.read_feather(path=datapath/'train.feather')
# y = np.array(df.target)
# dump(y, filename=datapath/'y.joblib')
# del df

y = load(datapath/'y.joblib')

# df.index.name = 'id'
# y_train = df.target
# features = [x for x in df.columns if x != 'target']
# X_train = df[features]
# # X.index.name = 'id'
# # y.index.name = 'id'
# X = np.array(X_train)
# y = np.array(y_train)

# del df, X_train, y_train

# load the Boruta-filtered green-zone 98 features (based on 200 iterations of the algo)
# train_source = '/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_filtered_green.joblib'
# X = load(train_source)

train_source = altdatapath/'train-WITH-KMeans_8cluster_ninit50_maxiter1000_rs42-AND-synthetic.feather' #'X_boruta_shap_200trials.feather'
exmodel_config['train_source'] = str(train_source)
X = pd.read_feather(path=train_source)
X = X.drop(['target'], axis=1)

# exmodel_config['feature_count'] = len(X.columns)
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

# exmodel_config['train_source'] = str(train_source)
# test_source = datapath/'test.feather'
# exmodel_config['test_source'] = str(test_source)
# X_test = pd.read_feather(path=test_source)
# X_test = X_test.iloc[:, 1:]
# X_test = np.array(X_test)

In [12]:
X.shape, y.shape

((1000000, 301), (1000000,))

In [13]:
type(X), type(y)

(pandas.core.frame.DataFrame, numpy.ndarray)

# Experiment setup

In [14]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

The best parameters up to present have been:

```python
best_xgboost_params = {
        'n_estimators': 3878,
        'max_depth': 4,
        'learning_rate': 0.024785857161974977,
        'reg_alpha': 26.867682044658245,
        'reg_lambda': 10.839759074147148,
        'subsample': 0.8208581489835881,
        'min_child_weight': 8.829122644339664,
        'colsample_bytree': 0.906420714280384,
        'gamma': 1.472322916021486
    }
```

These params get the following ROC_AUC scores on a 20% holdout for these dataset versions:

| Version | Feature Count | Valid ROC-AUC |
| ----- | ----- | ----- |
| original | 285 | 0.8572984856383443 |
| Boruta green-only | 98 | 0.8553163413048461 |
| Boruta green-and-blue | 109 | 0.8558487581638441 |
| Boruta with SHAP | 136 | 0.8566790062778752 |
| Original plus KMeans 8 plus synth | 301 | 0.8570855909847465 |

In [15]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=int(SEED), shuffle=True)
# # create wrappers for the training and validation partitions
# # train_pool = catboost.Pool(X_train, y_train)
# # valid_pool = catboost.Pool(X_valid, y_valid)

# # experimental parameters -- based off prev. best
# params = {
#     'n_estimators': 3878,
#     'max_depth': 4,
#     'learning_rate': 0.024785857161974977,
#     'reg_alpha': 26.867682044658245,
#     'reg_lambda': 10.839759074147148,
#     'subsample': 0.8208581489835881,
#     'min_child_weight': 8.829122644339664,
#     'colsample_bytree': 0.906420714280384,
#     'gamma': 1.472322916021486
# }

# # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
# model = XGBClassifier(
#     objective='binary:logistic',
#     verbosity=1,
#     tree_method='gpu_hist',
#     booster='gbtree', # not bothering with dart for time reasons
#     random_state=SEED,
# #         n_jobs=-1,
#     **params
# )    

# model.fit(X_train, y_train)
# # generate predictions
# preds = model.predict_proba(X_valid)[:,1]
# # rounds to the nearest integer, and the nearest even in case of _.5s

# # Evaluation
# valid_auc = roc_auc_score(y_valid, preds)
# print('ROC AUC Score of XGBoost =', valid_auc)

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:49:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC Score of XGBoost = 0.8570855909847465


In [15]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
       
    # else:
    #     print("No best study yet")
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
    # train_pool = catboost.Pool(X_train, y_train)
    # valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters -- based off prev. best
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 3500, 10000), # was 900-4500 for CPU
        'max_depth' : trial.suggest_int('depth', 3, 8),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.2),               
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 50),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 30),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
#         'booster': trial.suggest_categorical('boosting_type', ['gbtree', 'dart']),
        'min_child_weight': trial.suggest_uniform('min_child_weight', 0.001, 12),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_uniform('gamma', 0.1, 10)
    }  

    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = XGBClassifier(
        objective='binary:logistic',
        verbosity=1,
        tree_method='gpu_hist',
        booster='gbtree', # not bothering with dart for time reasons
        random_state=SEED,
#         n_jobs=-1,
        **params
    )    

    model.fit(X_train, y_train)
    # generate predictions
    preds = model.predict_proba(X_valid)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s

    # Evaluation
    valid_auc = roc_auc_score(y_valid, preds)
    print('ROC AUC Score of XGBoost =', valid_auc)
    wandb.log({'valid_auc': valid_auc,
              })

    return valid_auc

In [16]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-16-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find sweep_xgboost_20211020.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [17]:
study = optuna.create_study(direction = "maximize", 
                            sampler = TPESampler(seed=int(SEED)), 
                            study_name=f"xgboost-X_orig+KMeans8+synth_{datetime.now().strftime('%Y%m%d')}")

# study = load(studypath/f"optuna_xgboost_study_106trials_20211004.joblib")


[I 2021-10-20 21:08:23,088] A new study created in memory with name: xgboost-X_orig+KMeans8+synth_20211020


In [18]:
# import torch

In [19]:
import xgboost

In [20]:
# xgboost.core.XGBoostError?

In [21]:
# study.optimize?

In [22]:
for x in range(1, 500):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc], show_progress_bar=False, catch=(xgboost.core.XGBoostError,)) 
    dump(study, filename=studypath/f"optuna_xgboost_study_{x}trials_{datetime.now().strftime('%Y%m%d')}.joblib")
#     dump(study.best_trial.params, filename=datapath/f'optuna_lightgbm_study_best-thru-{x*5}trials_20210927.joblib')

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:08:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-10-20 21:16:37,181] Trial 0 finished with value: 0.8519012987970622 and parameters: {'n_estimators': 5934, 'depth': 8, 'learning_rate': 0.0483437145318464, 'reg_alpha': 0.6502468545951017, 'reg_lambda': 0.004994757081068292, 'subsample': 0.5779972601681014, 'min_child_weight': 0.6979452624062253, 'colsample_bytree': 0.9330880728874675, 'gamma': 6.051038616257767}. Best is trial 0 with value: 0.8519012987970622.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'n_estimators': 5934, 'depth': 8, 'learning_rate': 0.0483437145318464, 'reg_alpha': 0.6502468545951017, 'reg_lambda': 0.004994757081068292, 'subsample': 0.5779972601681014, 'min_child_weight': 0.6979452624062253, 'colsample_bytree': 0.9330880728874675, 'gamma': 6.051038616257767, 'value': 0.8519012987970622}.


ROC AUC Score of XGBoost = 0.8519012987970622


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[I 2021-10-21 08:31:51,753] Trial 55 finished with value: 0.8551605631113013 and parameters: {'n_estimators': 8950, 'depth': 4, 'learning_rate': 0.004715626782169541, 'reg_alpha': 0.0036018934890190207, 'reg_lambda': 10.64635101337637, 'subsample': 0.7028283632206299, 'min_child_weight': 4.760158184763182, 'colsample_bytree': 0.997538803763743, 'gamma': 1.1291914110307473}. Best is trial 48 with value: 0.8563981996999422.
wandb: WARNING Step must only increase in log calls.  Step 55 < 56; dropping {'n_estimators': 8950, 'depth': 4, 'learning_rate': 0.004715626782169541, 'reg_alpha': 0.0036018934890190207, 'reg_lambda': 10.64635101337637, 'subsample': 0.7028283632206299, 'min_child_weight': 4.760158184763182, 'colsample_bytree': 0.997538803763743, 'gamma': 1.1291914110307473, 'value': 0.8551605631113013}.


ROC AUC Score of XGBoost = 0.8551605631113013


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:31:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [ ]:
# dump(study, filename=datapath/f"optuna_xgboost_100trials-complete_{datetime.now().strftime('%Y%m%d')}.joblib")
# dump(study.best_trial.params, filename=datapath/f"optuna_lightgbm_all-500trials-best_{datetime.now().strftime('%Y%m%d')}.joblib")
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

Best on original dataset was:

```python
{'n_estimators': 3878,
 'depth': 4,
 'learning_rate': 0.024785857161974977,
 'reg_alpha': 26.867682044658245,
 'reg_lambda': 10.839759074147148,
 'subsample': 0.8208581489835881,
 'min_child_weight': 8.829122644339664,
 'colsample_bytree': 0.906420714280384,
 'gamma': 1.472322916021486}
```

In [ ]:
study.best_trial.params

In [ ]:
wandb.log({'xgboost_params': study.best_trial.params})
wandb.finish()

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)